<a href="https://colab.research.google.com/github/eteral00/UNT_CSCE5218_Project_DogsBreeds/blob/main/PMG_SubmissionVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting Google Drive**
You will need to add a shortcut to this Google shared folder to your own Google Drive to be able to access the dataset, the saved models, nad saved output statistics: \\
https://drive.google.com/drive/folders/1IypaLVub6Mk9A0AO_hN3wv5yEY04JNin

As well as going through this list of shared folders so that your drive recognized you have access to them (they are allshared as "has-link-can-view", but your GGDrive still need to recognize that you have the access) \\
https://drive.google.com/drive/folders/18QH_EOzL4halbKdHtDBil-De9Ia_Ys1d

https://drive.google.com/drive/folders/1Q2AhR6p28aD6J57ICYipuwvPgCgLoj1t

https://drive.google.com/drive/folders/1O75reXnX9LOT6LusUM1KUN6X8DlDor6S

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Handling Data**

## Decompress dataset

In [ ]:
# !unzip "/content/drive/MyDrive/low-resolution.zip" -d "/content/"
!unzip "/content/drive/MyDrive/CSCE5218/Dog_Breed_classification/Dog_Breed_dataset/Tshihua/low-resolution.zip" -d "/content/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/low-resolution/7449-n000128-teddy/n165759.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165760.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165761.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165762.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165763.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165764.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165765.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165766.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165767.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165768.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165769.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165770.jpeg  
  inflating: /content/low-resolution/7449-n000128-teddy/n165771.jpeg  
  inflating: /content/low-

In [ ]:
# !unzip "/content/drive/MyDrive/Dog_Breed_dataset/TrainValSplit.zip" -d "/content/"
!unzip "/content/drive/MyDrive/CSCE5218/Dog_Breed_classification/Dog_Breed_dataset/Tshihua/TrainValSplit.zip" -d "/content/"

Archive:  /content/drive/MyDrive/CSCE5218/Dog_Breed_classification/Dog_Breed_dataset/Tshihua/TrainValSplit.zip
   creating: /content/TrainAndValList/
  inflating: /content/TrainAndValList/train.lst  
  inflating: /content/TrainAndValList/validation.lst  


# Preprocessing data

In [ ]:
import argparse
import os
import shutil
import logging
import sys
import glob
from typing import Dict, Any, Set

def split_train_test(images_dir: str, output_dir: str, lst_path: str):
    with open(lst_path, "r") as f:
        lst: Set[str] = set([os.path.basename(line.strip()) for line in f.readlines()])

    for image_name in lst:
        image_path: str = glob.glob(os.path.join(images_dir, "*", image_name), recursive=True)[0]
        class_name: str = os.path.basename(os.path.dirname(image_path))

        dest_dir: str = os.path.join(output_dir, class_name)
        os.makedirs(dest_dir, exist_ok=True)

        logging.info(f"Copying {image_path} to {dest_dir}")
        shutil.copy(image_path, dest_dir)

In [ ]:
TSINGHUA_DOGS_ROOT_DIR = "/content/drive/MyDrive/CSCE5218/Dog_Breed_classification"
content_root = "/content"
images_dir: str = os.path.join(content_root, "low-resolution")
train_lst_path: str = os.path.join(content_root, "TrainAndValList", "train.lst")
test_lst_path: str = os.path.join(content_root, "TrainAndValList", "validation.lst")


In [ ]:
## Only run this once if putting data in shared folder on ggdrive

# #Prepare train set
# train_dir: str = os.path.join(TSINGHUA_DOGS_ROOT_DIR, "train")
# os.makedirs(train_dir, exist_ok=True)
# split_train_test(images_dir, output_dir=train_dir, lst_path=train_lst_path)

# #Prepare validation set
# test_dir: str = os.path.join(TSINGHUA_DOGS_ROOT_DIR, "test")
# os.makedirs(test_dir, exist_ok=True)
# split_train_test(images_dir, output_dir=test_dir, lst_path=test_lst_path)

In [ ]:
##If putting data in temporary runtime directory, need to rerun this every time

##Prepare train set
train_dir: str = os.path.join(content_root, "train")
os.makedirs(train_dir, exist_ok=True)
split_train_test(images_dir, output_dir=train_dir, lst_path=train_lst_path)

##Prepare test set
test_dir: str = os.path.join(content_root, "test")
os.makedirs(test_dir, exist_ok=True)
split_train_test(images_dir, output_dir=test_dir, lst_path=test_lst_path)

# **Define Neural Network Models**

## Defining ResNet with pretrained Resnet models

In [ ]:
import torch.nn as nn
from  torch.utils.model_zoo import load_url as load_state_dict_from_url



model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x1 = self.maxpool(x)

        x2 = self.layer1(x1)
        x3 = self.layer2(x2)
        x4 = self.layer3(x3)
        x5 = self.layer4(x4)

        x = self.avgpool(x5)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x1, x2, x3, x4, x5

##end ResNet class


def _resnet(arch, inplanes, planes, pretrained, progress, **kwargs):
    model = ResNet(inplanes, planes, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(**kwargs):
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained=False, progress=True, **kwargs)


def resnext101_32x8d(**kwargs):
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained=False, progress=True, **kwargs)

resnet_set = {"resnet18": resnet18, 
              "resnet34": resnet34, 
              "resnet50": resnet50,
              "resnet101": resnet101,
              "resnet152": resnet152,
              "resnext50_32x4d": resnext50_32x4d,
              "resnext101_32x8d": resnext101_32x8d
              }

## Defining PMG model

In [ ]:
import torch.nn as nn
import torch
    

class PMG(nn.Module):
    def __init__(self, model, feature_size, classes_num):
        super(PMG, self).__init__()

        self.features = model
        self.max1 = nn.MaxPool2d(kernel_size=56, stride=56)
        self.max2 = nn.MaxPool2d(kernel_size=28, stride=28)
        self.max3 = nn.MaxPool2d(kernel_size=14, stride=14)
        self.num_ftrs = 2048 * 1 * 1
        self.elu = nn.ELU(inplace=True)

        self.classifier_concat = nn.Sequential(
            nn.BatchNorm1d(1024 * 3),
            nn.Linear(1024 * 3, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        self.conv_block1 = nn.Sequential(
            BasicConv(self.num_ftrs//4, feature_size, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        self.conv_block2 = nn.Sequential(
            BasicConv(self.num_ftrs//2, feature_size, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier2 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        self.conv_block3 = nn.Sequential(
            BasicConv(self.num_ftrs, feature_size, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier3 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

    def forward(self, x):
        xf1, xf2, xf3, xf4, xf5 = self.features(x)

        # print("xf3: ", type(xf3))
        
        xl1 = self.conv_block1(xf3)
        xl2 = self.conv_block2(xf4)
        xl3 = self.conv_block3(xf5)
        
        xl1 = self.max1(xl1)
        xl1 = xl1.view(xl1.size(0), -1)
        xc1 = self.classifier1(xl1)

        xl2 = self.max2(xl2)
        xl2 = xl2.view(xl2.size(0), -1)
        xc2 = self.classifier2(xl2)

        xl3 = self.max3(xl3)
        xl3 = xl3.view(xl3.size(0), -1)
        xc3 = self.classifier3(xl3)
          
        x_concat = torch.cat((xl1, xl2, xl3), -1)
        x_concat = self.classifier_concat(x_concat)
        return xc1, xc2, xc3, x_concat
    
    
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size,
                              stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5,
                                 momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        # print("BasicConv: ", self)
        # print("x: ", type(x))
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

# **Define training function & utilities**

## Utilities

In [ ]:
import numpy as np
import random
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable



def cosine_anneal_schedule(t, nb_epoch, lr):
    cos_inner = np.pi * (t % (nb_epoch))  # t - 1 is used when t has 1-based indexing.
    cos_inner /= (nb_epoch)
    cos_out = np.cos(cos_inner) + 1

    return float(lr / 2 * cos_out)


def build_model(resnet_version, pretrain=True, require_grad=True):
    print('==> Building model: ' + resnet_version + "..")
    net = resnet_set[resnet_version](pretrained=pretrain)
    for param in net.parameters():
      param.requires_grad = require_grad
    net = PMG(net, 512, 200)

    return net


def model_info(model):  # Plots a line-by-line description of a PyTorch model
    n_p = sum(x.numel() for x in model.parameters())  # number parameters
    n_g = sum(x.numel() for x in model.parameters() if x.requires_grad)  # number gradients
    print('\n%5s %50s %9s %12s %20s %12s %12s' % ('layer', 'name', 'gradient', 'parameters', 'shape', 'mu', 'sigma'))
    for i, (name, p) in enumerate(model.named_parameters()):
        name = name.replace('module_list.', '')
        print('%5g %50s %9s %12g %20s %12.3g %12.3g' % (
            i, name, p.requires_grad, p.numel(), list(p.shape), p.mean(), p.std()))
    print('Model Summary: %g layers, %g parameters, %g gradients\n' % (i + 1, n_p, n_g))


def jigsaw_generator(images, n):
    l = []
    for a in range(n):
        for b in range(n):
            l.append([a, b])
    block_size = 448 // n
    rounds = n ** 2
    random.shuffle(l)
    jigsaws = images.clone()
    for i in range(rounds):
        x, y = l[i]
        temp = jigsaws[..., 0:block_size, 0:block_size].clone()
        jigsaws[..., 0:block_size, 0:block_size] = jigsaws[..., x * block_size:(x + 1) * block_size,
                                                y * block_size:(y + 1) * block_size].clone()
        jigsaws[..., x * block_size:(x + 1) * block_size, y * block_size:(y + 1) * block_size] = temp

    return jigsaws


def test(net, criterion, batch_size=3, testloader=None):
    net.eval()
    use_cuda = torch.cuda.is_available()
    test_loss = 0
    correct = 0
    correct_com = 0
    total = 0
    idx = 0
    device = torch.device("cuda:0")

    for batch_idx, (inputs, targets) in enumerate(testloader):
        idx = batch_idx
        if use_cuda:
            inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        output_1, output_2, output_3, output_concat= net(inputs)
        outputs_com = output_1 + output_2 + output_3 + output_concat

        loss = criterion(output_concat, targets)

        test_loss += loss.item()
        _, predicted = torch.max(output_concat.data, 1)
        _, predicted_com = torch.max(outputs_com.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        correct_com += predicted_com.eq(targets.data).cpu().sum()

        if batch_idx % 50 == 0:
            print('Testing| Step: %d | Loss: %.3f | Acc: %.3f%% (%d/%d) |Combined Acc: %.3f%% (%d/%d)' % (
            batch_idx, test_loss / (batch_idx + 1), 100. * float(correct) / total, correct, total, 100. * float(correct_com) / total, correct_com, total))

    test_acc = 100. * float(correct) / total
    test_acc_en = 100. * float(correct_com) / total
    test_loss = test_loss / (idx + 1)

    return test_acc, test_acc_en, test_loss


# Defining training function

In [ ]:
from __future__ import print_function
import os
from PIL import Image

import logging
import random
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn



def train(nb_epoch, batch_size, store_name, resume=False, start_epoch=0, model_path=None, trainloader=None, testloader=None, batch_size_test=3, in_net=None):
    # setup output
    exp_dir = store_name
    try:
        os.stat(exp_dir)
    except:
        os.makedirs(exp_dir)

    use_cuda = torch.cuda.is_available()
    print(use_cuda)


    # Model
    if in_net:
      net = in_net
    else:
      net = build_model(resnet_version='resnet50', pretrain=True, require_grad=True)

    if resume:
        # net = torch.load(model_path)
        net.load_state_dict(torch.load(model_path))

    netp = torch.nn.DataParallel(net, device_ids=[0])

    # GPU
    device = torch.device("cuda:0")
    net.to(device)
    # cudnn.benchmark = True
    CELoss = nn.CrossEntropyLoss()
    optimizer = optim.SGD([
        {'params': net.classifier_concat.parameters(), 'lr': 0.002},
        {'params': net.conv_block1.parameters(), 'lr': 0.002},
        {'params': net.classifier1.parameters(), 'lr': 0.002},
        {'params': net.conv_block2.parameters(), 'lr': 0.002},
        {'params': net.classifier2.parameters(), 'lr': 0.002},
        {'params': net.conv_block3.parameters(), 'lr': 0.002},
        {'params': net.classifier3.parameters(), 'lr': 0.002},
        {'params': net.features.parameters(), 'lr': 0.0002}

    ],
        momentum=0.9, weight_decay=5e-4)

    max_val_acc = 0
    lr = [0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.0002]
    for epoch in range(start_epoch, nb_epoch):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        train_loss1 = 0
        train_loss2 = 0
        train_loss3 = 0
        train_loss4 = 0
        correct = 0
        total = 0
        idx = 0
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            idx = batch_idx
            if inputs.shape[0] < batch_size:
                continue
            if use_cuda:
                inputs, targets = inputs.to(device), targets.to(device)
            inputs, targets = Variable(inputs), Variable(targets)

            # update learning rate
            for nlr in range(len(optimizer.param_groups)):
                optimizer.param_groups[nlr]['lr'] = cosine_anneal_schedule(epoch, nb_epoch, lr[nlr])

            # Step 1
            optimizer.zero_grad()
            inputs1 = jigsaw_generator(inputs, 8)
            # print("inputs1: ", type(inputs1))
            output_1, _, _, _ = netp(inputs1)
            loss1 = CELoss(output_1, targets) * 1
            loss1.backward()
            optimizer.step()

            # Step 2
            optimizer.zero_grad()
            inputs2 = jigsaw_generator(inputs, 4)
            _, output_2, _, _ = netp(inputs2)
            loss2 = CELoss(output_2, targets) * 1
            loss2.backward()
            optimizer.step()

            # Step 3
            optimizer.zero_grad()
            inputs3 = jigsaw_generator(inputs, 2)
            _, _, output_3, _ = netp(inputs3)
            loss3 = CELoss(output_3, targets) * 1
            loss3.backward()
            optimizer.step()

            # Step 4
            optimizer.zero_grad()
            _, _, _, output_concat = netp(inputs)
            concat_loss = CELoss(output_concat, targets) * 2
            concat_loss.backward()
            optimizer.step()

            #  training log
            _, predicted = torch.max(output_concat.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

            train_loss += (loss1.item() + loss2.item() + loss3.item() + concat_loss.item())
            train_loss1 += loss1.item()
            train_loss2 += loss2.item()
            train_loss3 += loss3.item()
            train_loss4 += concat_loss.item()

            if batch_idx % 50 == 0:
                print(
                    'Training| Step: %d | Loss1: %.3f | Loss2: %.5f | Loss3: %.5f | Loss_concat: %.5f | Loss: %.3f | Acc: %.3f%% (%d/%d)' % (
                    batch_idx, train_loss1 / (batch_idx + 1), train_loss2 / (batch_idx + 1),
                    train_loss3 / (batch_idx + 1), train_loss4 / (batch_idx + 1), train_loss / (batch_idx + 1),
                    100. * float(correct) / total, correct, total))
            if batch_idx % 500 == 0:
                ## save model every 500 steps
                net.cpu()
                torch.save(net.state_dict(), store_name + '/model_pmgnet.pth')
                net.to(device)

        train_acc = 100. * float(correct) / total
        train_loss = train_loss / (idx + 1)
        with open(exp_dir + '/results_train.txt', 'a') as file:
            file.write(
                'Training| Iteration %d | train_acc = %.5f | train_loss = %.5f | Loss1: %.3f | Loss2: %.5f | Loss3: %.5f | Loss_concat: %.5f |\n' % (
                epoch, train_acc, train_loss, train_loss1 / (idx + 1), train_loss2 / (idx + 1), train_loss3 / (idx + 1),
                train_loss4 / (idx + 1)))

        # if epoch < 5 or epoch >= 50:
        val_acc, val_acc_com, val_loss = test(net, CELoss, batch_size=batch_size_test, testloader=testloader) #batchsize_test=3
        if val_acc_com > max_val_acc:
            max_val_acc = val_acc_com
            net.cpu()
            torch.save(net.state_dict(), store_name + '/model_pmgnet_high.pth')
            torch.save(net.state_dict(), store_name + '/model_pmgnet.pth')
                # torch.save(net, store_name + '/model_pmgnet.pth')
            net.to(device)
            with open(exp_dir + '/results_test_high.txt', 'a') as file_h:
                file_h.write('Testing| Iteration %d, test_acc = %.5f, test_acc_combined = %.5f, test_loss = %.6f\n' % 
                        (epoch, val_acc, val_acc_com, val_loss)
                )
        else:
            net.cpu()
            torch.save(net.state_dict(), store_name + '/model_pmgnet.pth')
            # torch.save(net, store_name + '/model_pmgnet.pth')
            net.to(device)

        with open(exp_dir + '/results_test.txt', 'a') as file:
            file.write('Testing| Iteration %d, test_acc = %.5f, test_acc_combined = %.5f, test_loss = %.6f\n' % 
                        (epoch, val_acc, val_acc_com, val_loss)
                      )

    return (net, CELoss)


## Dataloaders

In [ ]:
import torch
import torchvision
from torchvision import transforms, models

##Setting parameters
batchSize_train = 16
batchSize_test = 1

##Transform Data
print("==> Preparing data..")
transform_train = transforms.Compose([
        transforms.Resize((550, 550)),
        transforms.RandomCrop(448, padding=8),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_test = transforms.Compose([
        transforms.Resize((550, 550)),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

==> Preparing data..


In [ ]:
##Data on shared folder on ggdrive
# rootPath = "/content/drive/MyDrive/CSCE5218/Dog_Breed_classification"

# trainset = torchvision.datasets.ImageFolder(root= rootPath + "/train", transform=transform_train)
# train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchSize_train, shuffle=True, num_workers=4)

# testset = torchvision.datasets.ImageFolder(root= rootPath + "/test", transform=transform_test)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize_test, shuffle=True, num_workers=4)

In [ ]:
##Alternatively, data on temporary runtime directory
rootPath = "/content"

trainset = torchvision.datasets.ImageFolder(root= rootPath + "/train", transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchSize_train, shuffle=True, num_workers=4)

testset = torchvision.datasets.ImageFolder(root= rootPath + "/test", transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batchSize_test, shuffle=True, num_workers=4)

# **Training Section**

## Calling train function

### Clean up GPU RAM

In [ ]:
torch.cuda.empty_cache() # clean memory in case GPU memory is fragmented

### Initial trainining
\* For first-start \\
\* Testing on reserved test set is done and recorded during training, at the end of each epoch

In [ ]:
# #Initial train



# #path to store output model and test statistics
# store_path = '/content/drive/MyDrive/CSCE5218/Dog_Breed_classification/output'

#temporary store on runtime directory
store_path = '/content/output'

# #initialize model
# pmgnet_in = build_model(resnet_version='resnet50', pretrain=True, require_grad=True) # not needed

#calling training
trained_net, trained_loss = train(nb_epoch=20,             # number of epoch
        batch_size=batchSize_train,         # batch size
        store_name=store_path,     # folder for output
        resume=False,          # resume training from checkpoint
        start_epoch=0,         # the start epoch number when you resume the training
        model_path=None,         # the saved model where you want to resume the training
        trainloader=train_loader,
        testloader=test_loader,
        batch_size_test=batchSize_test
        )

### Resumed training

In [ ]:
torch.cuda.empty_cache() # clean memory in case GPU memory is fragmented

In [ ]:
# #Resumed train

# print(store_path)

# #path to store output model and test statistics
store_path = '/content/drive/MyDrive/CSCE5218/Dog_Breed_classification/output'
# #path to load saved model from last run
modelPath = store_path + "/model_pmgnet.pth"

#temporary store on runtime directory
store_path = '/content/output'


pmgnet_in = build_model(resnet_version='resnet50', pretrain=True, require_grad=True)
# pmgnet_in.load_state_dict(torch.load(modelPath)) # not needed, reload done inside train function

# last epoch from previous run
resumeEpoch = 0


trained_net, trained_loss = train(nb_epoch=20,             # number of epoch
        batch_size=batchSize_train,         # batch size
        store_name=store_path,     # folder for output
        resume=True,          # resume training from checkpoint
        start_epoch=resumeEpoch,         # the start epoch number when you resume the training
        model_path=modelPath,         # the saved model where you want to resume the training
        trainloader=train_loader,
        testloader=test_loader,
        batch_size_test=batchSize_test,
        in_net=pmgnet_in
        )

==> Building model: resnet50..


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

True

Epoch: 5
Step: 0 | Loss1: 1.307 | Loss2: 0.86106 | Loss3: 0.44486 | Loss_concat: 0.16357 | Loss: 2.777 | Acc: 100.000% (16/16)
Step: 50 | Loss1: 1.093 | Loss2: 0.58100 | Loss3: 0.25516 | Loss_concat: 0.21227 | Loss: 2.141 | Acc: 96.814% (790/816)
Step: 100 | Loss1: 1.042 | Loss2: 0.54608 | Loss3: 0.23577 | Loss_concat: 0.20730 | Loss: 2.032 | Acc: 96.906% (1566/1616)
Step: 150 | Loss1: 1.027 | Loss2: 0.53027 | Loss3: 0.22888 | Loss_concat: 0.19834 | Loss: 1.985 | Acc: 97.268% (2350/2416)
Step: 200 | Loss1: 1.030 | Loss2: 0.53229 | Loss3: 0.22572 | Loss_concat: 0.19127 | Loss: 1.979 | Acc: 97.481% (3135/3216)
Step: 250 | Loss1: 1.032 | Loss2: 0.53611 | Loss3: 0.23095 | Loss_concat: 0.19405 | Loss: 1.993 | Acc: 97.435% (3913/4016)
Step: 300 | Loss1: 1.025 | Loss2: 0.53440 | Loss3: 0.22708 | Loss_concat: 0.18750 | Loss: 1.974 | Acc: 97.508% (4696/4816)
Step: 350 | Loss1: 1.015 | Loss2: 0.52960 | Loss3: 0.22272 | Loss_concat: 0.18340 | Loss: 1.950 | Acc: 97.507% (5476/5616)
Step: 400

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Testing| Step: 0 | Loss: 0.074 | Acc: 100.000% (1/1) |Combined Acc: 100.000% (1/1)
Testing| Step: 50 | Loss: 0.813 | Acc: 78.431% (40/51) |Combined Acc: 80.392% (41/51)
Testing| Step: 100 | Loss: 0.890 | Acc: 75.248% (76/101) |Combined Acc: 78.218% (79/101)
Testing| Step: 150 | Loss: 0.847 | Acc: 75.497% (114/151) |Combined Acc: 77.483% (117/151)
Testing| Step: 200 | Loss: 0.770 | Acc: 77.114% (155/201) |Combined Acc: 79.602% (160/201)
Testing| Step: 250 | Loss: 0.877 | Acc: 75.697% (190/251) |Combined Acc: 78.088% (196/251)
Testing| Step: 300 | Loss: 0.845 | Acc: 76.080% (229/301) |Combined Acc: 78.073% (235/301)
Testing| Step: 350 | Loss: 0.861 | Acc: 75.783% (266/351) |Combined Acc: 78.348% (275/351)
Testing| Step: 400 | Loss: 0.860 | Acc: 76.060% (305/401) |Combined Acc: 78.554% (315/401)
Testing| Step: 450 | Loss: 0.837 | Acc: 76.275% (344/451) |Combined Acc: 78.492% (354/451)
Testing| Step: 500 | Loss: 0.788 | Acc: 77.844% (390/501) |Combined Acc: 79.641% (399/501)
Testing| Step: